In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# データ読み込み
notepc_data = pickle.load(open("notepc_data.pkl", "br"))

# カラム
# 0: price
# 1: num_of_cores
# 2: clock
# 3: memory
# 4: disk
# 5: weight
# 6: months

In [ ]:
# 実際に使うデータを切り出し
# 価格は 1/100000スケールにて。価格もクロック数も、（ベクトルのままではなく）行列形式に変換
price = np.array([t[0] for t in notepc_data], dtype=np.float32) / 100000.
price.resize((price.size, 1))
clock = np.array([t[2] for t in notepc_data], dtype=np.float32)
clock.resize((clock.size, 1))

In [ ]:
clock.shape

In [ ]:
# データ可視化
plt.scatter(clock, price)
plt.xlabel("clock")
plt.ylabel("price")

<img src="new_network.png" width="400" style="float: left; margin: 5px;">

In [ ]:
# ※ このセルの内容を変更して、ネットワークに1層追加

# パラメータ、データ、モデルを用意

# パラメータ初期値
w1_ar = np.array([[-0.683, -0.833, -0.049]], dtype=np.float32)
b1_ar = np.array([0.499, 0.085, -0.654], dtype=np.float32)
w2_ar = np.random.random((3, 3)) * 2. - 1.
b2_ar = np.random.random((3,)) * 2. - 1.
w3_ar = np.array([[246071, 125136, 36801]], dtype=np.float32) / 100000.
b3_ar = np.array([116149], dtype=np.float32) / 100000.

# 上記の値を使い、variable定義
w1 = tf.Variable(w1_ar, dtype=tf.float32)
b1 = tf.Variable(b1_ar, dtype=tf.float32)
w2 = tf.Variable(w2_ar, dtype=tf.float32)
b2 = tf.Variable(b2_ar, dtype=tf.float32)
w3 = tf.Variable(w3_ar, dtype=tf.float32)
b3 = tf.Variable(b3_ar, dtype=tf.float32)

# データの入り口、placeholderを x (clock)、t (price) について定義
x_ph = tf.placeholder(tf.float32)
t_ph = tf.placeholder(tf.float32)


# == モデル定義 ==
# 1層目
h1 = tf.nn.sigmoid(x_ph * w1 + b1)
    
# 2層目
h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)

# 3層目
y = tf.matmul(h2, tf.transpose(w3)) + b3

# == コスト定義 ==
cost = 0.5 * tf.reduce_mean((y - t_ph) ** 2.)

# == 学習用のopを定義 ==
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.3)
train_op = trainer.minimize(cost, var_list=[w1, b1, w2, b2, w3, b3])

In [ ]:
# モデルのプロット用関数
x_val = np.linspace(1., 3.2, 101)
x_val.resize((x_val.size, 1))

def plot(scale=1.):        
    y_val = sess.run(y, feed_dict={x_ph: x_val})
    plt.plot(x_val, y_val * scale, color="orange")

    plt.scatter(clock, price * scale)
    plt.xlabel("clock")
    plt.ylabel("price")
    plt.ylim([(price *scale).min() - 0.15 * scale, (price * scale).max() + 0.15 * scale])

In [ ]:
# モデル可視化

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plot();

In [ ]:
# 実際の計算
history = list()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(20000):
        # 現状のコスト値を計算・記録、同時に学習も行う
        fd = {x_ph: clock, t_ph: price}
        cost_val, _ = sess.run([cost, train_op], feed_dict=fd)
        history.append(cost_val)

        # 進捗モニタリング
        if i % 1000 == 0:
            print("{:>6d}, cost: {:7.5f}".format(i, cost_val))

    # 結果として得られたモデルの可視化
    plot()

In [ ]:
# 学習曲線
plt.plot([t for t in history])
plt.xlabel("iterations")
plt.ylabel("cost")